In [1]:
# -*- coding: utf-8 -*-
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import logging
import os

# 로깅 설정 (노트북 출력 및 콘솔에 표시됨)
# 기존 핸들러 제거 (노트북에서 재실행 시 중복 로깅 방지)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 요청 헤더 (실제 브라우저처럼 보이도록 설정)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Wanted-Platform': 'web', # 필요할 수 있는 헤더
    'Wanted-Service': 'wanted', # 필요할 수 있는 헤더
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7', # 언어 설정 추가
}

logging.info("라이브러리 임포트, 로깅 및 헤더 설정 완료")

2025-04-17 16:10:25,778 - INFO - 라이브러리 임포트, 로깅 및 헤더 설정 완료


In [ ]:
# 직종 구분 선택
job_category = "total" # "total" # "backend" # "frontend"
if job_category == "total":
    job_category_url = "?"
elif job_category == "backend":
    job_category_url = "?job_ids=872"
elif job_category == "frontend":
    job_category_url = "?job_ids=669"
else:
    job_category_url = "?"


In [ ]:
# 02. 스크래핑 설정
scraped_data = []
offset_no = 0

# limit = 페이지 당 가져올 공고 수
# max_offset = 최대 가져올 공고 수
limit = 20
max_offset = 3000

api_base_url = f"https://www.wanted.co.kr/api/chaos/navigation/v1/results{job_category_url}"

# 기본 URL 파라미터
base_params = {
    'job_group_id': 518, # 개발 직군 전체
    'country': 'kr',
    'job_sort': 'job.latest_order', # 최신순 정렬
    'years': -1, # 경력 무관
    'locations': 'all', # 전체 지역
    'limit': limit,
    # 'offset'은 루프 내에서 설정됨
}

logging.info(f"스크래핑 설정 완료. 최대 offset: {max_offset}, 페이지당 공고 수: {limit}")

2025-04-17 16:10:25,796 - INFO - 스크래핑 설정 완료. 최대 offset: 600, 페이지당 공고 수: 20


In [4]:
# 03. 상세 정보 추출 함수 정의
def get_job_details(job_id):
    
    """
    주어진 job_id에 해당하는 상세 페이지 API에서 직무 상세 설명을 추출합니다.
    API 엔드포인트: https://www.wanted.co.kr/api/chaos/jobs/v4/{job_id}/details
    추출 필드: data.job.detail.main_tasks, data.job.detail.requirements, data.job.detail.preferred_points
    """

    api_url = f"https://www.wanted.co.kr/api/chaos/jobs/v4/{job_id}/details"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(api_url, headers=headers, timeout=15)
        response.raise_for_status()  # HTTP 오류 발생 시 예외 발생
        data = response.json()

        if 'data' in data and 'job' in data['data'] and 'detail' in data['data']['job']:
            detail = data['data']['job']['detail']
            main_tasks = detail.get('main_tasks', '')
            requirements = detail.get('requirements', '')
            preferred_points = detail.get('preferred_points', '')

            # 각 항목을 줄바꿈 두 번으로 연결하여 상세 정보 생성
            detail_text = "\n\n".join(filter(None, [main_tasks, requirements, preferred_points])).strip()
            return detail_text
        else:
            logging.warning(f"Detail section not found in API response for job ID {job_id}. URL: {api_url}. Response: {data}")
            return "상세 내용 없음 (API 응답에 Detail 섹션 부재)"

    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching details from API for job ID {job_id}: {e}. URL: {api_url}")
        return f"상세 내용 로드 실패 (API 요청 오류: {e})"
    except Exception as e:
        logging.error(f"Error parsing API response for job ID {job_id}: {e}", exc_info=True)
        return f"상세 내용 파싱 실패 (API 응답 오류: {e})"

# # --- 사용 예시 (테스트용, 실제 스크래핑 루프에서는 호출됨) ---
# test_job_id = 260432 # 테스트하려는 실제 공고 ID 입력
# test_details = get_job_details(test_job_id)
# print(f"--- 테스트 결과 (Job ID: {test_job_id}) ---")
# print(test_details)
# print(type(get_job_details(test_job_id)))
# print("--- 테스트 종료 ---")

In [5]:
# 04. 스크래핑 루프
logging.info("스크래핑 루프 시작...")

while offset_no < max_offset:
    # 현재 offset을 포함한 파라미터 설정
    current_params = base_params.copy()
    current_params['offset'] = offset_no

    logging.info(f"--- Offset {offset_no} 데이터 가져오는 중 ---")

    try:
        # API 요청 (타임아웃 20초)
        response = requests.get(api_base_url, params=current_params, headers=headers, timeout=20)
        response.raise_for_status() # 오류 발생 시 예외 발생
        json_data = response.json()

        if 'data' in json_data and json_data['data']:
            jobs = json_data['data']
            logging.info(f"Offset {offset_no}: {len(jobs)}개 공고 발견.")
            #

            for job in jobs:
                try:
                    company = job.get('company', {})
                    company_id = job.get('id')
                    company_name = company.get('name')
                    position = job.get('position')
                    # address = job.get('address', {}) # address 정보 가져오기
                    # location = address.get('location') # location 정보 추출
                    # district = address.get('district') # district 정보 추출
    
                    if not company_id:
                        logging.warning(f"ID가 없는 공고 건너<0xEB><0x9B><0x9C>. Position: {position}, Company: {company_name}")
                        continue
                    
                    # 상세 정보 가져오기 (정의된 함수 호출)
                    logging.info(f"  - 상세 정보 가져오는 중 (ID: {company_id}, Position: {position[:20]}...)") # 너무 길면 잘라서 로깅
                    skill = get_job_details(company_id)
                    # 각 상세 페이지 요청 사이에 1초 지연 (서버 부하 감소)
                    time.sleep(1)
    
                    job_info = {
                        'company': company_name,
                        'position': position,
                        'skill': skill,
                        # 'job_id': company_id, # 참고용으로 ID도 저장
                        # 'location': location, # location 정보 추가
                        # 'district': district # district 정보 추가
                    }
                    scraped_data.append(job_info)
    
                except Exception as e:
                    # 개별 공고 처리 중 오류 발생 시 로깅하고 계속 진행
                    logging.error(f"개별 공고 처리 중 오류 발생 (ID: {job.get('id')}): {e}", exc_info=True) # 상세 오류 스택 출력
                    continue

            # 다음 페이지로 이동
            offset_no += limit
            # API 목록 요청 사이에 1.5초 지연
            logging.info(f"Offset {offset_no - limit} 처리 완료. 1.5초 대기...")
            time.sleep(1.5)

        else:
            # API 응답에 'data'가 없거나 비어있는 경우
            logging.info(f"Offset {offset_no}에서 더 이상 데이터가 없거나 API 응답 구조 변경됨. 스크래핑 중단.")
            break

    except requests.exceptions.Timeout:
        logging.error(f"API 요청 시간 초과 (Offset: {offset_no}). 10초 후 재시도...")
        time.sleep(10)
        # continue # 현재 offset 재시도 위해 offset_no 증가시키지 않음 (루프 시작 시 재시도됨)

    except requests.exceptions.HTTPError as e:
        logging.error(f"HTTP 오류 발생 (Offset: {offset_no}): {e.response.status_code} - {e.response.reason}")
        if e.response.status_code == 429: # Too Many Requests
            logging.warning("요청 속도 제한(429) 감지됨. 60초 대기 후 재시도...")
            time.sleep(60)
            # continue # 재시도
        elif 400 <= e.response.status_code < 500:
            logging.error("클라이언트 오류 발생. 요청 파라미터나 헤더 확인 필요. 스크래핑 중단.")
            break
        elif e.response.status_code >= 500:
            logging.warning("서버 오류 발생. 30초 대기 후 재시도...")
            time.sleep(30)
            # continue # 재시도
        else:
            logging.error("알 수 없는 HTTP 오류. 스크래핑 중단.")
            break

    except requests.exceptions.RequestException as e:
        logging.error(f"API 요청 중 오류 발생 (Offset: {offset_no}): {e}")
        logging.info("네트워크 연결 확인 후 20초 대기...")
        time.sleep(20)
        # continue # 재시도

    except Exception as e:
        # 기타 예외 처리 (JSON 파싱 오류 등)
        logging.error(f"스크래핑 루프 중 예기치 않은 오류 발생 (Offset: {offset_no}): {e}", exc_info=True)
        logging.info("오류로 인해 스크래핑 중단.")
        break

logging.info(f"스크래핑 루프 종료. 총 {len(scraped_data)}개의 공고 데이터 수집 완료.")

# # (선택적) 수집된 데이터 개수 확인
# print(f"Total jobs collected: {len(scraped_data)}")
# # (선택적) 첫번째 데이터 샘플 확인
# if scraped_data:
#     print("\nSample Data:")
#     print(scraped_data[0])

2025-04-17 16:10:25,817 - INFO - 스크래핑 루프 시작...
2025-04-17 16:10:25,818 - INFO - --- Offset 0 데이터 가져오는 중 ---
2025-04-17 16:10:25,979 - INFO - Offset 0: 20개 공고 발견.
2025-04-17 16:10:25,979 - INFO -   - 상세 정보 가져오는 중 (ID: 276508, Position: [인공지능솔루션] Frontend 개...)
2025-04-17 16:10:27,108 - INFO -   - 상세 정보 가져오는 중 (ID: 230222, Position: [hourplace] React 프론...)
2025-04-17 16:10:28,222 - INFO -   - 상세 정보 가져오는 중 (ID: 256259, Position: 프론트엔드 엔지니어...)
2025-04-17 16:10:29,330 - INFO -   - 상세 정보 가져오는 중 (ID: 261544, Position: Frontend Engineer (R...)
2025-04-17 16:10:30,454 - INFO -   - 상세 정보 가져오는 중 (ID: 278600, Position: 웹 프론트엔드 개발자...)
2025-04-17 16:10:31,550 - INFO -   - 상세 정보 가져오는 중 (ID: 278596, Position: 프론트엔드 개발자 (3년 이상)...)
2025-04-17 16:10:32,682 - INFO -   - 상세 정보 가져오는 중 (ID: 151126, Position: 웹 프론트엔드 개발자...)
2025-04-17 16:10:33,837 - INFO -   - 상세 정보 가져오는 중 (ID: 202867, Position: Front-End Engineer...)
2025-04-17 16:10:34,984 - INFO -   - 상세 정보 가져오는 중 (ID: 261839, Position: [개발팀] Tech Inn

In [6]:
# 04. CSV 파일 생성
if not scraped_data:
    logging.warning("수집된 데이터가 없습니다. CSV 파일을 생성하지 않습니다.")
else:
    logging.info("수집된 데이터를 DataFrame으로 변환 중...")
    df = pd.DataFrame(scraped_data)

    # CSV 파일을 저장할 폴더 이름
    data_folder = 'data'

    # CSV 파일 이름 설정
    filename = f'data_wanted_{job_category}.csv'

    # 저장할 전체 경로 생성
    filepath = os.path.join(data_folder, filename)

    # 해당 폴더가 없으면 생성
    if not os.path.exists(data_folder):
        try:
            os.makedirs(data_folder)
            logging.info(f"'{data_folder}' 폴더를 생성했습니다.")
        except OSError as e:
            logging.error(f"'{data_folder}' 폴더 생성 중 오류 발생: {e}", exc_info=True)
            print(f"\n폴더 생성 실패: {e}")
    else:
        logging.info(f"'{data_folder}' 폴더가 이미 존재합니다.")

    try:
        # encoding='utf-8-sig' : Excel에서 한글 깨짐 방지 (BOM 포함 UTF-8)
        df.to_csv(filepath, index=False, encoding='utf-8-sig')
        logging.info(f"DataFrame이 '{filepath}'으로 성공적으로 저장되었습니다.")
        print(f"\n파일 저장 완료: {filepath}")
    except Exception as e:
        logging.error(f"DataFrame을 CSV로 저장하는 중 오류 발생: {e}", exc_info=True)
        print(f"\n파일 저장 실패: {e}")

2025-04-17 16:22:34,622 - INFO - 수집된 데이터를 DataFrame으로 변환 중...
2025-04-17 16:22:34,623 - INFO - 'data' 폴더가 이미 존재합니다.
2025-04-17 16:22:34,634 - INFO - DataFrame이 'data\data_wanted_frontend.csv'으로 성공적으로 저장되었습니다.



파일 저장 완료: data\data_wanted_frontend.csv


In [7]:
df_read = pd.read_csv(f"data/{filename}", encoding='utf-8-sig')
df_read

,company,position,skill
0,제논,[인공지능솔루션] Frontend 개발자,• React 기반 서비스 GenOS 관리자 대시보드 및 사용자 어플리케이션 요구기...
1,먼치팩토리,[hourplace] React 프론트엔드 개발자,• UI/UX 설계 및 구현: 사용자 친화적이고 직관적인 인터페이스를 React로 ...
2,펫프렌즈,프론트엔드 엔지니어,1. 펫프렌즈 모바일 APP/WEB 서비스 개발 및 기존 서비스 유지보수\n2. U...
3,메딜리티,Frontend Engineer (React & Flutter),[포지션 개요]\n• 최상의 사용자 경험을 만들기 위해 적합한 기술을 활용하여 서비...
4,에이든랩,웹 프론트엔드 개발자,• 인플루언서 마케팅 및 이커머스 모니터링 신규 서비스 프론트엔드 개발\n• 더 나...
...,...,...,...
584,휴맥스네트웍스,WiFi 소프트웨어 개발,• WiFi Driver Porting / 기능 개발 / 검증\n• Embedded...
585,제노소프트,C# WPF 개발자,• 영상솔루션 Windows 응용프로그램개발 (C# WPF)\n•\n\n• C# W...
586,유디아이디(UDID),임대형쇼핑몰 PHP 개발자,• 임대형 쇼핑몰서비스 PHP 개발 및 유지보수\n\n• 경력 3년 이상\n• PH...
587,베어런,프론트엔드 개발자 (3년~6년),• 헬스케어 산업 및 스포츠 산업 구인구직 서비스 상용화 모델의 개발 및 사용성 고...
